In [ ]:
import sys
sys.path.insert(1, f"../")
from utils import CremaDataset, LogMelTransform, train_model_teacher

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
ANNOTATIONS_FILE = 'labels.csv'
AUDIO_DIR = '../Crema-D'

traindataset = CremaDataset(ANNOTATIONS_FILE,
                            AUDIO_DIR,
                            data_portion = 'train',
                            transform = LogMelTransform)

valdataset = CremaDataset(ANNOTATIONS_FILE,
                          AUDIO_DIR,
                          data_portion = 'validation',
                          transform = LogMelTransform)

testdataset = CremaDataset(ANNOTATIONS_FILE,
                           AUDIO_DIR,
                           data_portion = 'test',
                           transform = LogMelTransform)

In [ ]:
from torch import nn

class CustomViT(nn.Module):
    def __init__(self):
        super().__init__()

        # convolutional encoder
        self.conv_encoder = nn.Sequential(
                                    nn.Conv2d(1, 16, 3, 1, 1),
                                    nn.InstanceNorm2d(16),
                                    nn.GELU(),
                                    nn.Conv2d(16, 32, 3, 1, 1),
                                    nn.InstanceNorm2d(32),
                                    nn.GELU(),
                                    nn.Conv2d(32, 64, 3, 1, 1),
                                    nn.InstanceNorm2d(64),
                                    nn.GELU(),
                                    nn.Conv2d(64, 32, 3, 1, 1),
                                    nn.InstanceNorm2d(32),
                                    nn.GELU(),
                                    nn.Conv2d(32, 16, 3, 1, 1),
                                    nn.InstanceNorm2d(16),
                                    nn.GELU(),
                                    nn.Conv2d(16, 1, 3, 1, 1),
                                    nn.InstanceNorm2d(1),
                                    nn.GELU())

    def forward(self, x):
        x = self.conv_encoder(x)
        print(x.shape)
        return x

In [24]:
from torch.utils.data import DataLoader

In [ ]:
# model
model = CustomViT().to(device)

# hyperparameters
lr = 1e-4
batch_size = 32
EPOCHS = 100
max_patience = 8
loss_fn = nn.CrossEntropyLoss()

# filename for parameters
model_path = 'train_results/teacher_params.pt'

# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# dataloaders
trainloader = DataLoader(traindataset, batch_size=batch_size, shuffle=True)
valloader = DataLoader(traindataset, batch_size=batch_size, shuffle=True)

# train model
results = train_model_teacher(model,
                              trainloader,
                              valloader,
                              optimizer,
                              loss_fn,
                              EPOCHS,
                              max_patience,
                              device,
                              model_path)

EPOCH: 1
torch.Size([64, 1, 128, 128])


RuntimeError: only batches of spatial targets supported (3D tensors) but got targets of dimension: 1